## Setup

In [1]:
!pip install xlrd


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\tusiu\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import sys
import os
from importlib import reload

# Add project directory to path
sys.path.append(os.path.join('.'))

from etl_pipeline_new import ETLPipeline
import data_sources_config

# Reload the config module to pick up changes
reload(data_sources_config)
from data_sources_config import get_data_sources_config

print("✓ Modules loaded successfully")

✓ Modules loaded successfully


## Initialize Pipeline

In [3]:
# Initialize ETL pipeline with PKD mapping file
pipeline = ETLPipeline(
    mapping_file_path=os.path.join('..', '..', 'data', 'mapowanie_pkd.xlsx')
)

print("✓ Pipeline initialized")

✓ Pipeline initialized


## Load and Process Data Sources

All data sources are configured in `data_sources_config.py`

In [4]:
# Get data sources configuration
data_sources = get_data_sources_config()

print(f"Found {len(data_sources)} data sources to process:\n")

# Process each data source
for source_config in data_sources:
    processor_class = source_config['processor_class']
    processor = processor_class(pipeline.pkd_mapper)
    
    pipeline.add_data_source(
        processor=processor,
        name=source_config['name'],
        **source_config['kwargs']
    )

print("\n✓ All data sources processed")

Found 1 data sources to process:

Processing Dane Miesięczne - liczba firm zarejestrowanych...
  ✓ Loaded 148 rows

✓ All data sources processed
  ✓ Loaded 148 rows

✓ All data sources processed


## Combine Data

In [5]:
# Combine all data sources into one fact table
combined_data = pipeline.combine_data()

print(f"✓ Combined data: {len(combined_data):,} rows")
print(f"  Years: {combined_data['rok'].min()} - {combined_data['rok'].max()}")
print(f"  Unique indicators: {combined_data['WSKAZNIK'].nunique():,}")
print(f"  Unique PKD codes: {combined_data['pkd_2025'].nunique():,}")

combined_data.head(10)

⚠ Invalid value: '31621*' | Source: Dane Miesięczne - liczba firm zarejestrowanych | Year: 2025 | PKD: OG | Indicator: Liczba firm zarejestrowanych Ogółem
✓ Combined data: 148 rows
  Years: 2018 - 2025
  Unique indicators: 1
  Unique PKD codes: 22
  Data sources: 1


,rok,pkd_2025,WSKAZNIK,wartosc,source
0,2021,OG,Liczba firm zarejestrowanych Ogółem,28698.0,Dane Miesięczne - liczba firm zarejestrowanych
1,2021,A,Liczba firm zarejestrowanych Ogółem,413.0,Dane Miesięczne - liczba firm zarejestrowanych
2,2021,B,Liczba firm zarejestrowanych Ogółem,19.0,Dane Miesięczne - liczba firm zarejestrowanych
3,2021,C,Liczba firm zarejestrowanych Ogółem,1940.0,Dane Miesięczne - liczba firm zarejestrowanych
4,2021,D,Liczba firm zarejestrowanych Ogółem,169.0,Dane Miesięczne - liczba firm zarejestrowanych
5,2021,E,Liczba firm zarejestrowanych Ogółem,54.0,Dane Miesięczne - liczba firm zarejestrowanych
6,2021,F,Liczba firm zarejestrowanych Ogółem,5654.0,Dane Miesięczne - liczba firm zarejestrowanych
7,2021,G,Liczba firm zarejestrowanych Ogółem,4851.0,Dane Miesięczne - liczba firm zarejestrowanych
8,2021,H,Liczba firm zarejestrowanych Ogółem,1541.0,Dane Miesięczne - liczba firm zarejestrowanych
9,2021,I,Liczba firm zarejestrowanych Ogółem,655.0,Dane Miesięczne - liczba firm zarejestrowanych


## Build Dimension Tables

In [6]:
# Build dimension tables and map indices
(
    fact_table,
    wskaznik_dictionary,
    pkd_dictionary,
    pkd_typ_dictionary
) = pipeline.build_dictionaries(combined_data)

print("✓ Dimension tables built")

✓ Dimension tables built


## Inspect Results

In [7]:
# Fact table (KPI values)
print("=== FACT TABLE ===")
print(f"Rows: {len(fact_table):,}")
print(f"Columns: {list(fact_table.columns)}")
display(fact_table.head(10))

=== FACT TABLE ===
Rows: 148
Columns: ['rok', 'wartosc', 'source', 'WSKAZNIK_INDEX', 'PKD_INDEX']


=== FACT TABLE ===
Rows: 148
Columns: ['rok', 'wartosc', 'source', 'WSKAZNIK_INDEX', 'PKD_INDEX']


,rok,wartosc,source,WSKAZNIK_INDEX,PKD_INDEX
0,2021,28698.0,Dane Miesięczne - liczba firm zarejestrowanych,0,1756
1,2021,413.0,Dane Miesięczne - liczba firm zarejestrowanych,0,1741
2,2021,19.0,Dane Miesięczne - liczba firm zarejestrowanych,0,1742
3,2021,1940.0,Dane Miesięczne - liczba firm zarejestrowanych,0,1743
4,2021,169.0,Dane Miesięczne - liczba firm zarejestrowanych,0,1744
5,2021,54.0,Dane Miesięczne - liczba firm zarejestrowanych,0,1745
6,2021,5654.0,Dane Miesięczne - liczba firm zarejestrowanych,0,1746
7,2021,4851.0,Dane Miesięczne - liczba firm zarejestrowanych,0,1747
8,2021,1541.0,Dane Miesięczne - liczba firm zarejestrowanych,0,1748
9,2021,655.0,Dane Miesięczne - liczba firm zarejestrowanych,0,1749


In [8]:
# WSKAZNIK dimension
print("=== WSKAZNIK DIMENSION ===")
print(f"Total indicators: {len(wskaznik_dictionary):,}")
display(wskaznik_dictionary.head(20))

=== WSKAZNIK DIMENSION ===
Total indicators: 1


=== WSKAZNIK DIMENSION ===
Total indicators: 1


,WSKAZNIK_INDEX,WSKAZNIK
0,0,Liczba firm zarejestrowanych Ogółem


In [9]:
# PKD dimension
print("=== PKD DIMENSION ===")
print(f"Total PKD codes: {len(pkd_dictionary):,}")
display(pkd_dictionary.head(20))

=== PKD DIMENSION ===
Total PKD codes: 1,764


=== PKD DIMENSION ===
Total PKD codes: 1,764


,PKD_INDEX,symbol,nazwa,TYP_INDEX
0,0,01,"UPRAWY ROLNE, CHÓW I HODOWLA ZWIERZĄT, ŁOWIECT...",1
1,1,01.1,Uprawy rolne inne niż wieloletnie,2
2,2,01.11,"Uprawa zbóż innych niż ryż, roślin strączkowyc...",3
3,3,01.11.Z,"Uprawa zbóż innych niż ryż, roślin strączkowyc...",4
4,4,01.12,Uprawa ryżu,3
5,5,01.12.Z,Uprawa ryżu,4
6,6,01.13,"Uprawa warzyw, włączając melony oraz uprawa ro...",3
7,7,01.13.Z,"Uprawa warzyw, włączając melony oraz uprawa ro...",4
8,8,01.14,Uprawa trzciny cukrowej,3
9,9,01.14.Z,Uprawa trzciny cukrowej,4


In [10]:
# PKD Type dimension
print("=== PKD TYPE DIMENSION ===")
print(f"Total types: {len(pkd_typ_dictionary):,}")
display(pkd_typ_dictionary)

=== PKD TYPE DIMENSION ===
Total types: 6


=== PKD TYPE DIMENSION ===
Total types: 6


,TYP_INDEX,typ
0,0,SEKCJA
1,1,DZIAŁ
2,2,GRUPA
3,3,KLASA
4,4,PODKLASA
5,5,OGÓŁEM


## Data Quality Checks

In [11]:
# Check for null values
print("=== NULL VALUE ANALYSIS ===")
null_counts = fact_table.isnull().sum()
null_percentages = (null_counts / len(fact_table) * 100).round(2)

for col in fact_table.columns:
    if null_counts[col] > 0:
        print(f"{col}: {null_counts[col]:,} ({null_percentages[col]}%)")
    else:
        print(f"{col}: ✓ No nulls")

=== NULL VALUE ANALYSIS ===
rok: ✓ No nulls
wartosc: 1 (0.68%)
source: ✓ No nulls
WSKAZNIK_INDEX: ✓ No nulls
PKD_INDEX: ✓ No nulls


In [12]:
# Check data types
print("=== DATA TYPES ===")
print(fact_table.dtypes)
print("\nValue types in 'wartosc' column:")
print(fact_table['wartosc'].apply(type).value_counts())

=== DATA TYPES ===
rok                 int64
wartosc           float64
source             object
WSKAZNIK_INDEX      int64
PKD_INDEX           int64
dtype: object

Value types in 'wartosc' column:
wartosc
<class 'float'>    148
Name: count, dtype: int64


## Save Results

In [13]:
# Save all tables to CSV files
pipeline.save_results(
    fact_table=fact_table,
    wskaznik_dict=wskaznik_dictionary,
    pkd_dict=pkd_dictionary,
    pkd_typ_dict=pkd_typ_dictionary,
    output_dir=os.path.join('..', '..', 'results-pipeline')
)


✓ All tables saved to ..\..\results-pipeline/
  - Fact table: 148 rows
  - WSKAZNIK dictionary: 1 indicators
  - PKD dictionary: 1,764 codes
  - PKD type dictionary: 6 types

  - Fact table: 148 rows
  - WSKAZNIK dictionary: 1 indicators
  - PKD dictionary: 1,764 codes
  - PKD type dictionary: 6 types


## Summary

### Output Files

- **kpi-value-table.csv**: Fact table with all KPI values
- **wskaznik_dictionary.csv**: Dimension table for indicators
- **pkd_dictionary.csv**: Dimension table for PKD codes
- **pkd_typ_dictionary.csv**: Dimension table for PKD types

### Schema

**Fact Table:**
- `rok` (int): Year
- `PKD_INDEX` (int): Foreign key to pkd_dictionary
- `WSKAZNIK_INDEX` (int): Foreign key to wskaznik_dictionary
- `wartosc` (Decimal/None): KPI value

**WSKAZNIK Dictionary:**
- `WSKAZNIK_INDEX` (int): Primary key
- `WSKAZNIK` (str): Indicator name

**PKD Dictionary:**
- `PKD_INDEX` (int): Primary key
- `symbol` (str): PKD code
- `nazwa` (str): PKD name
- `TYP_INDEX` (int): Foreign key to pkd_typ_dictionary

**PKD Type Dictionary:**
- `TYP_INDEX` (int): Primary key
- `typ` (str): Type name (Sekcja, Dział, etc.)